# Deploy LVA & Rocket to an IoT Edge Device
Unless specified, all command cells below will be running on the development/deployment machine (not on the edge device)

!!! IMPORTANT !!! like in all previous sections, also in this section, for a succesful deployement, be sure you read each node description carefully and run each nodel cell one by one.

## Get global variables
We will read the previously stored global variables and pick a Rocket pipeline for video analytics. We have pre-compiled Rocket images of seven different pipelines. Please set *pipeline* below to your desired pipeline ID. Pipeline descriptions are also included in [Rocket-features-and-pipelines.pdf](https://aka.ms/Microsoft-Rocket-LVA-features-and-pipelines.pdf).

Pipeline ID | Pipeline Descriptions
--- | --- | ---
1 | Background subtraction (BGS)-based early filtering -> ONNX Tiny YOLOv3 -> ONNX YOLOv3 ([slide #7](https://aka.ms/Microsoft-Rocket-LVA-features-and-pipelines.pdf#page=7))
2 | BGS-based early filtering -> ONNX YOLOv3 ([slide #8](https://aka.ms/Microsoft-Rocket-LVA-features-and-pipelines.pdf#page=8))
3 | ONNX YOLOv3 on every frame ([slide #9](https://aka.ms/Microsoft-Rocket-LVA-features-and-pipelines.pdf#page=9))
4 | ONNX Tiny YOLOv3 on every frame ([slide #10](https://aka.ms/Microsoft-Rocket-LVA-features-and-pipelines.pdf#page=10))
5 | BGS-based counting ([slide #11](https://aka.ms/Microsoft-Rocket-LVA-features-and-pipelines.pdf#page=11))
6 | ONNX YOLOv3 object detection on each frame ([slide #12](https://aka.ms/Microsoft-Rocket-LVA-features-and-pipelines.pdf#page=12))
7 | ONNX Tiny YOLOv3 object detection on each frame ([slide #13](https://aka.ms/Microsoft-Rocket-LVA-features-and-pipelines.pdf#page=13))

In [ ]:
import sys
sys.path.append('../../../utilities/video-analysis/notebooks/common')

from env_variables import *

In [ ]:
set_key(envPath, "CONTAINER_IMAGE_NAME", "rocket")

## Setup Local Folders on the Edge Device
Create two folders on the "IoT Edge Device", the device where we want to deploy our solution.

First folder, name it "input", will contain a sample video file that we will use to play in an IP Camera simulator. We will use IP Camera simulator (RTSPsimulator) for those who dont have IP camera but want to run this samples.
Second folder, name it "output", will be empty but will be used to store debug outputs (intermediate inference results such as video frames with bounding box marks as wrote out as jpeg files.)
Both folder must have read/write access for all users. Below are the commands that we use to create them and set their access permissions. Run the below commands **on the IoT Edge Device**

```
sudo mkdir -p /lvarocket/input
sudo mkdir /lvarocket/output
```

Now download the "[rocket-sample.mkv](https://aka.ms/lva-rocket-videosample)" video file into "/lvarocket/input"  
```
sudo wget https://aka.ms/lva-rocket-videosample -O /lvarocket/input/sample.mkv
```

Finally we set these folder's access permissions.  
```
sudo chmod 777 -R /lvarocket
```

when you list the folder contents, it should have read/write access to all as sample below:  

```
researcher@iotedge:~/Desktop/aiext$ ls -la /lvarocket/input/
total 4524
drwxrwxrwx 2 root root    4096 May 19 18:34 .
drwxrwxrwx 4 root root    4096 May 19 18:29 ..
-rwxrwxrwx 1 mksa mksa 4620607 May 19 18:19 sample.mkv
```

### Set .env file for deployment

We will use an IoT Edge Device Module Deployment Manifest file (deployment.lvaedge_rocketserver_rtspsim.template.json) which has parameter placeholders that we will replace with our service names, credentials etc. that we created in the previous sections. This placeholder parameters are auto set by VSCode using the **".env"** file. So below cell will update the .env file with the required parameters.

In [ ]:
set_key(envPath, "INPUT_VIDEO_FOLDER_ON_DEVICE", "/lvarocket/input")
set_key(envPath, "OUTPUT_VIDEO_FOLDER_ON_DEVICE", "/lvarocket/output")

Next, we need to set Rocket pipeline OPTIONS.  

OPTIONS | Descriptions
--- | ---
line | Line configuration (ROCKET_LINE). URI of the [line configuration file](#prepare-video-input-and-line-configuration) that defines the area of interest. ROCKET_LINE is required for all pipelines except rocket-ppl5.
cat | Object category (ROCKET_CAT). A string with object categories of interest separated by space. Categories should match the [MS-COCO classes](https://gist.github.com/AruniRC/7b3dadd004da04c80198557db5da4bda), e.g., car or handbag. 
sfactor | Frame sampling factor (ROCKET_RFACTOR, int). Uniformly samples 1 frame out of ROCKET_SFACTOR frames for processing; the other frames are discarded. The sampling parameter (ROCKET_SFACTOR) should be appropriately set depending on the frame rate of the video stream and content in the video. If the video consists of faster moving objects, a lower ROCKET_SFACTOR is recommended, otherwise higher values can be efficiently used.
rfactor | Frame resolution factor (ROCKET_RFACTOR, double value between 0 and 1). Downsizes the input frame's height and width by a factor of ROCKET_RFACTOR before processing the frame. As above, the resolution factor should be set depending on the original resolution of the frame and content in the video. If the scene is cluttered with many objects, then the ROCKET_RFACTOR should be set closer to 1.
bsize | Frame buffer size (ROCKET_BUFFER_SIZE, int): Size of the window of frames in history over which the DNNs are executed when an object of interest crosses the line of interest. Larger values of ROCKET_BUFFER_SIZE may result in over-counting when the objects in the video are moving fast.
uptran, downtran | Transition durations (ROCKET_ UP_TRANSITION and ROCKET_ DOWN_TRANSITION): Durations for which occupancy of the objects over the line of interest is measured. We recommend smaller values for UP_TRANSITION and DOWN_TRANSITION when the video's frame rate is low. Default values for ROCKET_ UP_TRANSITION and ROCKET_ DOWN_TRANSITION are 1 and 1 for a video stream of 5 frames/second. The lower the values of ROCKET_ UP_TRANSITION and ROCKET_ DOWN_TRANSITION, the higher the chances of over-counting.

In [ ]:
pipeline = "1"
category = "car"
linecfg = "https://aka.ms/linesample"
sfactor = "1"
rfactor = "1"
bsize = "20"
uptran = "4"
downtran = "10"

In [ ]:
set_key(envPath, 'ROCKET_PIPELINE', 'pipeline=' + pipeline)
set_key(envPath, 'ROCKET_CAT', 'cat=' + category)
set_key(envPath, 'ROCKET_LINE', 'line=' + linecfg)
set_key(envPath, 'ROCKET_SFACTOR', 'sfactor=' + sfactor)
set_key(envPath, 'ROCKET_RFACTOR', 'rfactor=' + rfactor)
set_key(envPath, 'ROCKET_BUFFERSIZE', 'bsize=' + bsize)
set_key(envPath, 'ROCKET_UPTRAN', 'uptran=' + uptran)
set_key(envPath, 'ROCKET_DOWNTRAN', 'downtran=' + downtran)

Finally, we need to copy the .env file to Rocket notebook folder for deployment. 

In [ ]:
from shutil import *
copy('../../../utilities/video-analysis/notebooks/common/.env', './.env')

Now, our IoT Edge Device module deployment manifest file (deployment.lvaedge_rocketserver_rtspsim.template.json) and deployment manifest placeholder variables (.env) are ready to be used.

## Deploy LVA and Rocket Modules

Follow these steps:

1. Now right click on the file named "deployment.lvaedge_rocketserver_rtspsim.template.json". In the pop-up menu, click on the "Create IoT Edge Deployment Manifest". In this step, VS code will auto read the contents of ".env" file and uses the values of the variables to write over some place holder variables inside the "deployment.lvaedge_rocketserver_rtspsim.template.json". After this replacement, new version of the file with the name "deployment.lvaedge_rocketserver_rtspsim.amd64.json" will be placed under the newly created "config" folder in the same working directory.  
<img src="documents/_deployment_modules_1.png" width=600px/>  

2. Right click on the file "/config/deployment.lvaedge_rocketserver_rtspsim.amd64.json", In the pop-up menu, click on the "Create Deployment for Single Device"  
    a. If it is the first time using current IoT Hub service that was created in the previous sections, VSCode will ask you to enter "IoT Hub Connection String", you can use the value of "iotHubConnString" key, which can be located in the ".env" file.  
    b. "Create Deployment for Single Device" command will open a window on top edge of the VSCode and will ask you the IoT Edge Device name that you want to make this deployment. Since you entered the IoT Hub name in step (a), it will automatically find the device names and list for you. Select this device in the drop down list.  
<img src="documents/_deployment_modules_2.png" width=600px/>  

3. Depending on your Internet speed, in seconds to minutes, modules will be pulled from the cloud and deployed into your edge device. In the VSCode, under "AZURE IOT HUB" panel, you can see the modules that are uploaded on to your edge device by refreshing.  
<img src="documents/_iotedgedevice_view.png" width=400px/>  

As in the above screen shot you must see "running" word near each module name (see region 3 in the above screenshot)  

Or you can run the following command in a terminal window on Iot Edge Device:  
```
sudo iotedge list
```

and the result of the above command will look like something similar to below with all module status are "running":  
```
researcher@iotedge:~/Desktop/aiext$ sudo iotedge list
[sudo] password for mksa: 
NAME             STATUS           DESCRIPTION      CONFIG
rocket           running          Up 6 minutes     mcr.microsoft.com/lva-utilities/rocket:1.0
edgeAgent        running          Up 7 minutes     mcr.microsoft.com/azureiotedge-agent:1.0
edgeHub          running          Up 6 minutes     mcr.microsoft.com/azureiotedge-hub:1.0
lvaEdge          running          Up 6 minutes     lvacontainers.azurecr.io/lvaedge:1.0.01227.756
rtspsim          running          Up 6 minutes     mcr.microsoft.com/lva-utilities/rtspsim-live555:1.2
```

At this point we succesfully deployed  modules into Iot Edge Device.  

If not all of the modules are running properly after deployment, please run the following command on the **IoT Edge Device** to restart the IoT Edge Runtime. It could take 1-2 minutes to have IoT Edge Runtime up and running in stable mode. 
```
sudo systemctl restart iotedge
```

## Deploy / Manage Media Graph on LVA edge module
At this point of the sample, we should have all required modules deployed to edge device, up and running.  

Now we will deploy Media graph into the "lvaEdge" module. Here "lvaEdge" is the name that we assign to the LVA module inside the module deployment template json file. Media graph is also a json file that defines a media flow pipeline from media sources to the target sinks and any anytics processes inbetween. Organization of the media graph file consists of following modules:  
```
{
    "@apiVersion": "1.0",

    "name": "SampleMediaGraphOrganization",

    "properties":{

        "parameters": ...

        "sources": ...

        "processes": ...

        "sinks": ...
    }
}
```

In the above media graph json file, apart from sources, processors and sinks, parameters section is the critical part of the media graph that defines all required parameters to run the media graph. In this section we define the addresses of the media sources, their access credentials and parameters related to the process modules like fps rate, sensitivity of motion detection module etc. Each media graph may have different media flow i.e. one may have motion detection processor while the other may not. User may modify this media graph according to problem that they want to solve in their scenarios.  



### Sample Media Graph
In this sample project we use a sample media graph file [topology.json](../../../../MediaGraph/topologies/httpExtension/topology.json). This media graph file dictates lvaEdge to ingest a video stream from a single IP camera, send decoded frames at **frameRate** to an AI module (i.e., Rocket). AI module analyses the stream and than the result (json format inference result) is sent to IoT Hub.

### Setting Parameters in Media Graph
There are two ways to set the parameters in the media graph:  
1. Update the default values of the parameters in "topology.json", which is not the prefered way.  
2. Note the parameter names and data types in "topology.json", and override their values in the deployment command that we will show in the next section.
  

### Deploy the Media Graph with Custom parameters
Below we will explain the following operations:
1. Setting (deploying) Media Graph Topology on the IoT Edge Device's lvaEdge module
2. Creating a Media Topolgy Instance from the one set in step 1
3. Activate the Topology instance that is set in step 2

Also we will show:
- How to list and delete the already set Graph Topologies on the Device
- How to list and delete the already set Graph Instances on the Device
- How to acticate / deactivate the Instances

#### Install prerequisites
We will use a simple python script to deploy our Media Graph json into the "lvaEdge" module. Which is a process of sending a file into IoT Edge Device over Internet through IoT Edge Hub service. To do so, we need to install following Microsoft developed generic Python packages that helps for interaction with IoT Hub Services.

!Important: As mentioned in the first section of this tutorial, you must use the right pip command (pip or pip3) depending on your Python installation. This sample is tested with > Python3.6 version and here we use pip3 command to install the packages into our python3 environment.

In [ ]:
!pip3 install azure-iot-device
!pip3 install azure-iot-hub

#### Import Helper Python class to deploy Media Graphs
Following line will import the custom Python class that we wrote to help us deploying Media edge graphs. After import, we will instantiate a graph manager object with IoT Hub, IoT Edge Device and Graph API version details.

In [ ]:
from graph_manager import GraphManager

moduleId = "lvaEdge" # Must be same as the name that we assigned to LVA module in the "deployment.lvaedge_rocketserver_rtspsim.template.json" file
operationsApiVersion = "1.0"  # Must be same as the version number in the "topology.json" file

graphManager = GraphManager(iotHubConnString, iotDeviceId, moduleId, operationsApiVersion)

#### List existing Graph Topologies and Instances


In [ ]:
# List topologies
response = graphManager.GenericCall("GraphTopologyList", {})
print(response)

In [ ]:
# List instances
response = graphManager.GenericCall("GraphInstanceList", {})
print(response)

#### Set Topology
Below code will deploy "topology.json" file into the lvaEdge module running on our IoT Edge Device.

In [ ]:
# You can define a topologyFile or a topologyUrl that you want to deploy into the module. Here we point to our sample Media Graph Topology File.
operationParams = {
                    "topologyFile": "../../../../MediaGraph/topologies/httpExtension/topology.json"
                    }

In [ ]:
# Set Graph Topology
response = graphManager.GraphTopologySet(operationParams)
print(response)

#### Set Topology Instance

Below code will set the values of the parameters mentioned in [topology.json](../../../../MediaGraph/topologies/httpExtension/topology.json) and will create a Topology Instance. If you modify the topology, add new parameters etc. you should update the below parameter list accordigly.  

! Important:  
- Be sure that the value of "topologyName" parameter below and the same parameter in the "topology.json" file to be the same.  
- Since we do not have physical IP camera set for this sample, we use a virtual IP camera simulator. In the above steps, with module deployment manifest, we already deployed a simulator module which plays "sample.mkv" video file as it played from an IP camera. So in the below parameter list, the parameter "rtspUrl" points to the URL address of this IoT Module (rtsp://rtspsim:554) with the full path of the video file that we want to play. You can replace this parameter's value with full RTSP address of your phyical IP camera.  
- rtspUsername and rtspPassword are dummy values for our simulator because it doesnt require any authentication. But if your source stream (i.e. phyical IP Camera) requires authentication, than put appropriate values for these parameters.  
- "hubSinkOutputName" parameter sets the IoT Hub channel name which ingest inference results into IoTHub.

In [ ]:
mediaGraphTopologyParameters = {
          "name": "Sample-Graph-1",
          "properties": {
            "topologyName": "InferencingWithHttpExtension",
            "description": "Sample graph description",
            "parameters": [
              {
                "name": "rtspUrl",
                "value": "rtsp://rtspsim:554/media/sample.mkv"
              },
              {
                "name": "rtspUserName",
                "value": "username"
              },
              {
                "name": "rtspPassword",
                "value": "password"
              },
              {
                "name": "inferencingUrl",
                "value": "http://rocket:7788/api/ImageItems"
              },
              {
                "name": "inferencingUserName",
                "value": "username"
              },
              {
                "name": "inferencingPassword",
                "value": "password"
              },
              {
                "name": "imageEncoding",
                "value": "jpeg"
              },
              {
                "name": "imageScaleMode",
                "value": "pad"
              },
              {
                "name": "frameWidth",
                "value": "720"
              },
              {
                "name": "frameHeight",
                "value": "400"
              },
              {
                "name": "frameRate",
                "value": "30"
              }
            ]
          }
        }

After setting above parameters as in Python dictionary data structure, now we can set an instance of previously deployed topology on the edge device with the above custom parameters.

In [ ]:
# Set topology instance
graphManager.GenericCall("GraphInstanceSet", mediaGraphTopologyParameters)
print(response)

#### Activate Topology Instance
Activates the Topology instance that we set in the previous node.

! Important:  
- Be sure to set the below name parameter to exact same topology instance name that we used in the previous node.

In [ ]:
# Activate topology instance
operationParams = {
                    "name": "Sample-Graph-1"
                    }

graphManager.GenericCall("GraphInstanceActivate", operationParams)
print(response)

This concludes the deployment of LVA and Rocket. Please move on to [output_format_and_interpretations.md](./output_format_and_interpretations.md) to check out the output. To remove the deployment, please follow the instructions below. 

## Remove the deployment

### De-Activate Topology Instance
In case you do not want to run the topology, at any time you can de-activates it. Below node will de-activate your Media Graph. Which means it will not be running anymore!

! Important:  
- Be sure to set the below name parameter to exact same topology instance name that we used in the previous node.


In [ ]:
# De-Activate topology instance
operationParams = {
                    "name": "Sample-Graph-1"
                    }

graphManager.GenericCall("GraphInstanceDeactivate", operationParams)
print(response)

### Delete Graph Topology Instance
Below code is just a sample to show how to delete previously deployed instance.

In [ ]:
# Delete instance
operationParams = {
                    "name": "Sample-Graph-1"
                    }

graphManager.GenericCall("GraphInstanceDelete", operationParams)
print(response)

### Delete Graph Topology
Below code is just a sample to show how to delete previously deployed toplogy.


! Important:  
- Be sure to set the below name parameter to exact same topology name that you want to delete

In [ ]:
# Delete topology
operationParams = {
                    "name": "InferencingWithHttpExtension"
                    }

graphManager.GenericCall("GraphTopologyDelete", operationParams)
print(response)

### Reset the IoT Edge Device  
To reset the IoT Edge Device, you can upload a deployment manifest that consist of only two system module (IoTHub, IoTEdge).  

1. Right click on the "deployment.reset.json" file under `live-video-analytics/utilities/video-analysis/notebooks/common/`  
<img src="documents/_deployment_reset.png" width=500px/>  

2. Click on "Create Deployment for Single Device"  
    a. If it is the first time using current IoT Hub service that was created in the previous sections, VSCode will ask you to enter "IoT Hub Connection String", you can use the value of "iotHubConnString" key, which can be located in the ".env" file.  
    b. "Create Deployment for Single Device" command will open a window on top edge of the VSCode and will ask you the IoT Edge Device name that you want to make this deployment. Since you entered the IoT Hub name in step (a), it will automatically find the device names and list for you. For this sample we have only a single device which we name it with the value of variable "iotDeviceId" in the second section where we defined the variables. Select this device in the drop down list.  
<img src="documents/_select_edge_device.png" width=500px/>  